In [1]:

"""
@author: pickle0412
"""

import numpy as np
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn
import torchvision
import torchvision.models as models
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
%matplotlib inline
from numpy import moveaxis
from skimage import io, transform
from torch.autograd import Variable
from PIL import Image
import pickle
import os
import glob
import pandas as pd
import random
import cv2
import plotly
# import bokeh
from sklearn.metrics import accuracy_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import warnings
warnings.filterwarnings("ignore")

plt.ion()

In [2]:
squeezenet = models.squeezenet1_1(pretrained=True)
for param in squeezenet.parameters():
    param.requires_grad=False

squeezenet

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace)
    )
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0

In [3]:
squeezenet.classifier[1]=torch.nn.Conv2d(512,512,kernel_size=(1,1),stride=(1,1))
squeezenet = nn.Sequential(squeezenet,
                    torch.nn.ReLU(),
                    torch.nn.Dropout(p=0.3),
                    torch.nn.Linear(512, 128),
                    torch.nn.ReLU(),
                    torch.nn.Dropout(p=0.2),
                    torch.nn.Linear(128, 10))

In [4]:
def load_train_dataset():
    data_path = 'AUCdata/v2_cam1_cam2_ split_by_driver/Camera 1/train/'
    train_dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        transform=torchvision.transforms.Compose([
        transforms.Resize(size=(100, 100)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    )
    return train_dataset

def load_test_dataset():
    data_path = 'AUCdata/v2_cam1_cam2_ split_by_driver/Camera 1/test/'
    test_dataset = torchvision.datasets.ImageFolder(
        root = data_path,
        transform = transforms.ToTensor()
    )
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size = 16,
        num_workers = 4,
        shuffle = False
    )
    return test_loader

In [5]:
dataset = load_train_dataset()
batch_size = 16
validation_split = .2
shuffle_dataset = True
random_seed= 42

dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler)

testLoader = load_test_dataset()

In [6]:
if torch.cuda.is_available():
    squeezenet.cuda()

In [7]:
criterion = nn.CrossEntropyLoss()

In [8]:
learning_rate = 0.001

optimizer = torch.optim.Adam(squeezenet.parameters(), lr = learning_rate)

In [9]:
def train(model,dataloaderList,epochs, device = 'cpu',saveName=None):
    
    trainloader=dataloaderList[0]
    valloader=dataloaderList[1]

    valAccList=[]
    
    print('Start Training...')
    for epoch in range(epochs):
        running_loss=0
        
        for i_batch, (images, labels) in enumerate(trainloader):
            
            images=Variable(images,requires_grad=True).to(device)
            labels=Variable(labels).to(device)
            
            optimizer.zero_grad()
            
            outputs=model(images)

            loss=criterion(outputs,labels)
            
            loss.backward()
            
            optimizer.step()
            
            running_loss += loss.item()
            
            if i_batch%100 == 99:
                print('[%2d,%5d] Training Loss: %.3f'  % 
                      (epoch+1,i_batch+1,running_loss/100))
                running_loss = 0

               
        print('Validating the model')
        
        with torch.no_grad():
            valPreds=[]
            valLabels=[]
            for i_batch, (images, labels) in enumerate(valloader):
                
                images= images.to(device)
                labels= labels.numpy()
                
                preds=model(images)
                preds=preds.cpu().numpy()
                
                pred_labels=np.argmax(preds,axis=-1)
                valPreds.append(pred_labels)
                valLabels.append(labels)
            
            valPreds=np.concatenate(valPreds)
            valLabels=np.concatenate(valLabels)
            size=valPreds.shape[0]
            running_corrects=np.sum(valPreds==valLabels)
        
            acc=float(running_corrects)/size
            valAccList.append(acc)
            print('Accuracy after epoch %2d is %.3f' % (epoch+1,acc))
            torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, 'Model_weights/'+saveName+'.pt')
    
    print("Finished Training")
    print("Saving Model")
#     torch.save(model.state_dict(),'Model_weights/'+saveName+'.pt')
    
    return model,valAccList

In [ ]:
squeezenet_1, valAccList_1 = train(squeezenet, [train_loader, validation_loader], 15, device, 'squeezenet')

In [ ]:
with open('squeezenet_val_acc_list', 'wb') as f:
    pickle.dump(valAccList_1, f)

In [10]:
model = squeezenet
checkpoint = torch.load('Model_weights/squeezenet.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
squeezenet.eval()

Sequential(
  (0): SqueezeNet(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2))
      (1): ReLU(inplace)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      (3): Fire(
        (squeeze): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
        (squeeze_activation): ReLU(inplace)
        (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
        (expand1x1_activation): ReLU(inplace)
        (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (expand3x3_activation): ReLU(inplace)
      )
      (4): Fire(
        (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
        (squeeze_activation): ReLU(inplace)
        (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
        (expand1x1_activation): ReLU(inplace)
        (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (expand3x3_activation): ReLU(inplace)
  

In [11]:
with torch.no_grad():
            valPreds=[]
            valLabels=[]
            valAccList = []
            for i_batch, (images, labels) in enumerate(validation_loader):
                
                images= images.to(device)
                labels= labels.numpy()
                
                preds=model(images)
                preds=preds.cpu().numpy()
                
                pred_labels=np.argmax(preds,axis=-1)
                valPreds.append(pred_labels)
                valLabels.append(labels)
            
            valPreds=np.concatenate(valPreds)
            valLabels=np.concatenate(valLabels)
            size=valPreds.shape[0]
            running_corrects=np.sum(valPreds==valLabels)
        
            acc=float(running_corrects)/size
            valAccList.append(acc)
            print('Accuracy after 15 epochs is %.3f' % (acc))

Accuracy after 15 epochs is 0.904
